# Тестовые задания. Каримов Ильдар 

## Задание №2 (Pandas, Numpy):

**Дано:** динамические данные по дебиту газа.

**Формат исходных данных:** 
- Date&Time – дата и время замера, 
- qg_sc – объемный дебит газа в м3\сут.

**Задача:**
Написать скрипт (на Python) для обнаружения выбросов в динамических данных по
дебиту газа. Данный скрипт должен читать исходные данные, добавлять столбец «Outlier» со
значениями 1 – выброс, 0 – не выброс, и записывать результат в файл Excel.
На основе написанного скрипта необходимо получить результат для 4 предоставленных
примеров.

**Замечания:** необходимо учесть, что данные динамические, то есть в них есть тренды.

**Идея:**
1. Проанализировать отклонения значения в точке от тренда в скользящем окне.
2. Часто при поиске выбросов пользуются правилом трех сигм.

## Решение: 

In [1]:
# Импортируем необходимые библиотеки

import pandas as pd
import numpy as np

In [2]:
# прочитаем файл и сохраним результат в датафрейме df
df = pd.read_excel('C:/Users/Задание № 2 (данные).xlsx', sheet_name='№2')

df.head() # выведем первые 5 строк таблицы на экран 

,Date&Time,qg_sc
0,2021-01-01 00:59:59,332.48
1,2021-01-01 01:59:59,333.65
2,2021-01-01 02:59:59,332.22
3,2021-01-01 03:59:59,333.52
4,2021-01-01 04:59:59,331.99


In [3]:
df['date'] = df['Date&Time'].dt.date # преобразуем временной тип 

In [4]:
# берем среднее арифметическое и среднее квадратическое отклонение за дату 

df_grouped = df.groupby('date').agg({'qg_sc': ['mean', 'std']}) 

In [5]:
df_grouped.head() # выведем первые 5 строк таблицы на экран 

qg_sc           
                  mean        std
date                             
2021-01-01  332.740833   0.775258
2021-01-02  333.399583   0.808291
2021-01-03  350.752917  13.497392
2021-01-04  361.490417   0.966650
2021-01-05  364.903333   4.439674

In [6]:
df_grouped.columns = ['_'.join(col) for col in df_grouped.columns.values] # уберем "этажность" названий столбцов 

In [7]:
df_grouped = df_grouped.reset_index() # уберем индекс 

In [8]:
df = df.merge(df_grouped) # объеденим таблицы 

In [12]:
# добавим новый столбец Outlier со значениями 1 – выброс, 0 – не выброс, при поиске выбросов пользуемся правилом трех сигм

df['Outlier'] = (
                ~(df.qg_sc > (df.qg_sc_mean - 3 * df.qg_sc_std)) & (df.qg_sc < (df.qg_sc_mean + 3 * df.qg_sc_std))).astype(
        int)

In [10]:
df[['Date&Time', 'qg_sc', 'Outlier']].head() # выведем первые 5 строк итоговой таблицы на экран 

,Date&Time,qg_sc,Outlier
0,2021-01-01 00:59:59,332.48,0
1,2021-01-01 01:59:59,333.65,0
2,2021-01-01 02:59:59,332.22,0
3,2021-01-01 03:59:59,333.52,0
4,2021-01-01 04:59:59,331.99,0


In [11]:
df[df['Outlier'] == 1] # выведем стороки с выбрасами 

,Date&Time,qg_sc,date,qg_sc_mean,qg_sc_std,Outlier
431,2021-01-18 23:59:58.995,352.06,2021-01-18,355.405833,1.032493,1
551,2021-01-23 23:59:58.995,284.94,2021-01-23,352.027083,14.418553,1
640,2021-01-27 16:59:58.995,303.09,2021-01-27,313.345000,3.355388,1
1305,2021-02-24 09:59:58.995,296.97,2021-02-24,309.700000,2.837686,1


## Скрипт: 

In [ ]:
import pandas as pd
import argparse

def main(path):
    print(path)
    df = pd.read_excel(path)
    df['date'] = df['Date&Time'].dt.date
    df_grouped = df.groupby('date').agg({'qg_sc': ['mean', 'std']})
    df_grouped.columns = ['_'.join(col) for col in df_grouped.columns.values]
    df_grouped = df_grouped.reset_index()
    df = df.merge(df_grouped)
    df['Outlier'] = (
                ~(df.qg_sc > (df.qg_sc_mean - 3 * df.qg_sc_std)) & (df.qg_sc < (df.qg_sc_mean + 3 * df.qg_sc_std))).astype(
        int)
    df[['Date&Time', 'qg_sc', 'Outlier']].to_excel(path, index=False)


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Outliers detector')
    parser.add_argument('filename')

    args = parser.parse_args()
    main(args.filename)

Скачать файл скрипта в формате Py по ссылке - https://disk.yandex.ru/d/L285kzBU8xblXg 